In [1]:
"""Preprocessing for news features."""

import os
os.chdir("c:/Users/gufer/OneDrive/Documentos/FIAP/Fase_05/ML_Engineer_Datathon/src/features")

import pandas as pd
import re

pd.set_option('display.max_colwidth', None) 
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

from utils import concatenate_csv_to_df
from constants import (
    news_template_path, 
    news_num_csv_files,
    #news_dtypes,
)

os.chdir("c:/Users/gufer/OneDrive/Documentos/FIAP/Fase_05/ML_Engineer_Datathon/")

In [17]:
def pre_process_news() -> pd.DataFrame:
    
    # Concatena CSVs
    df_news = concatenate_csv_to_df(news_template_path, news_num_csv_files)
    
    # Converte para datetimes nas colunas de data
    df_news['issued'] = pd.to_datetime(df_news['issued'])
    df_news['modified'] = pd.to_datetime(df_news['modified'])

    # Criar colunas separadas para data e hora
    df_news['issuedDate'] = df_news['issued'].dt.date
    df_news['issuedTime'] = df_news['issued'].dt.time
    df_news['modifiedDate'] = df_news['modified'].dt.date
    df_news['modifiedTime'] = df_news['modified'].dt.time
 
    # Extrai o miolo relevante da URL
    regex = r'(?<=g1\.globo\.com\/)(.*?)(?=\/noticia)'
    df_news['urlExtracted'] = df_news['url'].str.extract(regex)
    
    # Função para identificar localidades
    def check_local(url):
        if pd.notna(url):
            # Padrão para localidades: duas letras, barra, nome da localidade
            regex = re.compile(r'^[a-z]{2}/[a-z-]+')
            match = regex.match(url)
            if match:
                return match.group()  # Retorna apenas a parte da localidade
        return None

    # Função para identificar temas
    def check_theme(url):
        localidade = check_local(url)
        if pd.notna(url):
            if localidade:
                # Se houver uma localidade, o tema é o restante da URL após a localidade
                tema = url.replace(localidade, '').lstrip('/')
                return tema if tema else None
            else:
                # Se não houver localidade, a URL inteira é o tema
                return url
        return None

    # Aplicar as funções ao DataFrame
    df_news['local'] = df_news['urlExtracted'].apply(check_local)
    df_news['localState'] = df_news['local'].str.split('/').str[0]
    df_news['localRegion'] = df_news['local'].str.split('/').str[1]
    df_news['theme'] = df_news['urlExtracted'].apply(check_theme)
    df_news['themeMain'] = df_news['theme'].str.split('/').str[0]
    df_news['themeSub'] = df_news['theme'].str.split('/').str[1]
    
    return df_news

df_news = pre_process_news()

In [21]:
df_news["themeSub"].value_counts()[:10]

themeSub
2022              2272
coronavirus       2137
musica            1970
2020              1471
2018              1330
agronegocios      1249
eleicoes          1176
cinema             875
tecnologia         764
ucrania-russia     517
Name: count, dtype: int64

In [4]:
df_news['urlExtracted'].tail(200)

55403    sp/vale-do-paraiba-regiao/especial-publicitario/atibaia-e-regiao-convention-e-visitors-bureau/turismo-atibaia-e-regiao
55404    sp/vale-do-paraiba-regiao/especial-publicitario/atibaia-e-regiao-convention-e-visitors-bureau/turismo-atibaia-e-regiao
55405                                                                                             sp/presidente-prudente-regiao
55406                                                                                             sp/presidente-prudente-regiao
55407                                                                                             sp/presidente-prudente-regiao
55408                                                                                        sp/sao-jose-do-rio-preto-aracatuba
55409                                                                                        sp/sao-jose-do-rio-preto-aracatuba
55410                                                                                        sp/sao-jose

In [5]:
# Ideia de feature: distância entre a data da notícia e a data que o usuário consumiu a notícia

In [6]:
df_news.columns

Index(['page', 'url', 'issued', 'modified', 'title', 'body', 'caption',
       'issuedDate', 'issuedTime', 'modifiedDate', 'modifiedTime',
       'urlExtracted', 'local', 'theme'],
      dtype='object')

In [7]:
df_news.info()

<class 'pandas.core.frame.DataFrame'>
Index: 255603 entries, 0 to 55602
Data columns (total 14 columns):
 #   Column        Non-Null Count   Dtype              
---  ------        --------------   -----              
 0   page          255603 non-null  object             
 1   url           255603 non-null  object             
 2   issued        255603 non-null  datetime64[ns, UTC]
 3   modified      255603 non-null  datetime64[ns, UTC]
 4   title         255603 non-null  object             
 5   body          255603 non-null  object             
 6   caption       255603 non-null  object             
 7   issuedDate    255603 non-null  object             
 8   issuedTime    255603 non-null  object             
 9   modifiedDate  255603 non-null  object             
 10  modifiedTime  255603 non-null  object             
 11  urlExtracted  250510 non-null  object             
 12  local         173222 non-null  object             
 13  theme         77288 non-null   object             

In [8]:
df_news["url"].head(100)



0                                                   http://g1.globo.com/am/amazonas/noticia/2022/06/18/caso-bruno-e-dom-3o-suspeito-tem-prisao-temporaria-decretada-pela-justica-do-am.ghtml
1                           http://g1.globo.com/pa/santarem-regiao/noticia/2019/06/20/linguajar-dos-santarenos-e-diferenciado-e-cheio-de-identidade-egua-tu-nao-vai-ser-leso-de-perder.ghtml
2                                                                            http://g1.globo.com/mundo/noticia/2022/07/08/ex-premie-shinzo-abe-morre-apos-ser-baleado-no-japao-diz-nhk.ghtml
3                                                 http://g1.globo.com/politica/noticia/2021/09/09/relator-no-stf-fachin-vota-contra-marco-temporal-para-demarcacao-de-terras-indigenas.ghtml
4                                         http://g1.globo.com/politica/noticia/2021/09/15/apos-2-votos-pedido-de-vista-suspende-julgamento-no-stf-sobre-demarcacao-de-terras-indigenas.ghtml
5                                      http://g1.globo.